In [1]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [10]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Set the path to the main directory containing subdirectories with images
main_dir = r'C:\Users\Plaksha\3D Objects\DL\fma_img\fma_img'

# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 32


In [11]:
# Set the path to the main directory containing subdirectories with images
main_dir = r'C:\Users\Plaksha\3D Objects\DL\fma_img\fma_img'

In [12]:
# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 32


In [13]:

# Define the number of images for train, validation, and test sets
train_size = 23
validation_size = 3
test_size = 2

In [14]:
# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use subset parameter for training data
)



Found 230 images belonging to 10 classes.


In [15]:
validation_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use subset parameter for validation data
)

Found 50 images belonging to 10 classes.


In [16]:
# Load the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])


In [17]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# Train the model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

# Print accuracy and other performance metrics
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss:", history.history['val_loss'][-1])



Epoch 1/20
8/8 [==============================] - 2s 238ms/step - loss: 2.1359e-04 - accuracy: 1.0000 - val_loss: 1.9670 - val_accuracy: 0.7200
Epoch 2/20
8/8 [==============================] - 2s 218ms/step - loss: 2.1051e-04 - accuracy: 1.0000 - val_loss: 2.0093 - val_accuracy: 0.7200
Epoch 3/20
8/8 [==============================] - 2s 217ms/step - loss: 2.0945e-04 - accuracy: 1.0000 - val_loss: 2.0121 - val_accuracy: 0.7200
Epoch 4/20
8/8 [==============================] - 2s 220ms/step - loss: 2.1056e-04 - accuracy: 1.0000 - val_loss: 1.9690 - val_accuracy: 0.7200
Epoch 5/20
8/8 [==============================] - 2s 208ms/step - loss: 2.0688e-04 - accuracy: 1.0000 - val_loss: 1.9959 - val_accuracy: 0.7200
Epoch 6/20
8/8 [==============================] - 2s 217ms/step - loss: 1.9890e-04 - accuracy: 1.0000 - val_loss: 2.0243 - val_accuracy: 0.7200
Epoch 7/20
8/8 [==============================] - 2s 216ms/step - loss: 1.9624e-04 - accuracy: 1.0000 - val_loss: 2.0080 - val_accuracy:

In [19]:
# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to calculate embeddings
def calculate_embeddings(image_path):
    img_array = preprocess_image(image_path)
    embeddings = model.predict(img_array)
    return embeddings



In [20]:
# Function to open file dialog and get the image path
def browse_file():
    file_path = filedialog.askopenfilename()
    image_path_var.set(file_path)
    if file_path:
        img = Image.open(file_path)
        img = img.resize((200, 200), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img



In [21]:
# Function to match embeddings and determine class
def match_embeddings():
    image_path = image_path_var.get()
    if image_path:
        uploaded_embeddings = calculate_embeddings(image_path)
        min_distance = float('inf')
        min_class = None
        for subdir in os.listdir(main_dir):
            subdir_path = os.path.join(main_dir, subdir)
            for img_file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, img_file)
                embeddings = calculate_embeddings(img_path)
                distance = np.linalg.norm(uploaded_embeddings - embeddings)
                if distance < min_distance:
                    min_distance = distance
                    min_class = subdir
        result_var.set(f"The song clip belongs to class: {min_class}")



In [22]:
# Create tkinter window
root = tk.Tk()
root.title("Image Classification")

# Create widgets
browse_button = tk.Button(root, text="Upload Image", command=browse_file)
browse_button.pack(pady=10)

image_label = tk.Label(root)
image_label.pack(pady=10)

image_path_var = tk.StringVar()
image_path_entry = tk.Entry(root, textvariable=image_path_var, state='readonly')
image_path_entry.pack(pady=10)

classify_button = tk.Button(root, text="Classify Image", command=match_embeddings)
classify_button.pack(pady=10)

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack(pady=10)

# Function to close the tkinter window
def close_window():
    root.destroy()

# Bind closing event to close_window function
root.protocol("WM_DELETE_WINDOW", close_window)

# Run the tkinter event loop
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Plaksha\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Plaksha\AppData\Local\Temp\ipykernel_7356\1162370698.py", line 7, in browse_file
    img = img.resize((200, 200), Image.ANTIALIAS)
                                 ^^^^^^^^^^^^^^^
AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 26ms/step
